# Tuning of dSPN neuromodulation

## Setup network with dSPN population
Create a network of disconnected dSPN neurons where each morphology key / parameter key combination is represented.

Here we use the ```setup_network``` function in InputTuning to create the network.

In [1]:
import os
from snudda.input.input_tuning import InputTuning

snudda_data = "../../../../../BasalGangliaData/data/"
network_path = os.path.join("..", "networks", "dspn_modulation")
input_tuning = InputTuning(network_path, snudda_data=snudda_data)

#neurons_path = os.path.join("$DATA", "neurons", "striatum")
neurons_path = os.path.join("..", "data")

input_tuning.setup_network(neurons_path=neurons_path, 
                           num_replicas=1,
                           neuron_types="dspn",
                           reaction_diffusion_file="../data/JSON/robert_reaction_diffusion.json",
                           morphology_key="m22be6817",
                           network_random_seed=1234)
input_tuning = None

Found 1 neuron models in ../data/dspn
Skipping neuron type SBML
Skipping neuron type dspn_rxd
Skipping neuron type dspn_no_rxd
Skipping neuron type mechanisms
Skipping neuron type JSON
Writing network config file to ../networks/dspn_modulation/network-config.json
Reading SNUDDA_DATA=../../../../../BasalGangliaData/data/ from ../networks/dspn_modulation/network-config.json
Generating 10928 points for data/mesh/InputTestMesh.obj
Filtering, keeping inside points: 9341 / 10217
neuron_name = 'str_dspn_e150602_c1_D1_mWT_0728MSN01_v20211026', num = 1, neuron_path = '../data/dspn/str-dspn-e150602_c1_D1-mWT-0728MSN01-v20211026'
Reading SNUDDA_DATA=../../../../../BasalGangliaData/data/ from ../networks/dspn_modulation/network-config.json
No d_view specified, running distribute neurons in serial
No connections specified in connectivity_distribution.
No file ../networks/dspn_modulation/pruning_merge_info.json


## Generate synaptic input for the neuron population

Setup glutamate input, and GABA input. Also generate dopamine input.

| time (s) | glutamate (Hz) | GABA (Hz) | DA (Hz) |
| --- | --- | --- | --- |
| 1 - 3 | 10 | 5 | 0 |
| 5 - 7 | 10 | 5 | 5 |
| 9 - 11 | 10 | 5 | 10 |

In [2]:
from snudda import Snudda
snd = Snudda(network_path=network_path)
snd.setup_input(input_config="input.json")

Setting up inputs, assuming input.json exists
Reading SNUDDA_DATA=../../../../../BasalGangliaData/data/ from ../networks/dspn_modulation/network-config.json
Writing input spikes to ../networks/dspn_modulation/input-spikes.hdf5
Reading SNUDDA_DATA=../../../../../BasalGangliaData/data/ from ../networks/dspn_modulation/network-config.json
Writing spikes to ../networks/dspn_modulation/input-spikes.hdf5
stop_parallel disabled, to keep pool running.

Execution time: 0.1s


## Simulate

In [3]:
duration = 11 # 4.5
simulation_config = "dspn_experiment_config.json"
exec_cmd = f"mpirun -n 6 snudda simulate {network_path} --time {duration} --simulation_config {simulation_config}" 
print(exec_cmd)
# os.system(exec_cmd)

mpirun -n 6 snudda simulate ../networks/dspn_modulation --time 11 --simulation_config dspn_experiment_config.json


## Plot PKA level

## Determine parameters:

KIR: ```mod_pka_g_min```, ```mod_pka_g_max```, ```mod_pka_g_half```, ```mod_pka_g_slope```

Kaf: ```mod_pka_g_min```, ```mod_pka_g_max```, ```mod_pka_g_half```, ```mod_pka_g_slope```, ```mod_pka_shift_min```, ```mod_pka_shift_max```, ```mod_pka_shift_half```, ```mod_pka_shift_slope```


## Plot the PKA activation curves

In [ ]:
from snudda.utils import SnuddaLoadSimulation

output_file = os.path.join(network_path, "simulation", "dspn-output.hdf5")
nd = SnuddaLoadSimulation(output_file)
time = nd.get_time()
data_pka = nd.get_data("PKA", 0)[0][0]
data_da = nd.get_data("DA", 0)[0][0]

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(time, data_da, label="DA")
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
# plt.legend()
plt.title("DA")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(time, data_pka, label="PKA")
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
# plt.legend()
plt.title("PKA")
plt.show()